In [2]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
#from statsmodels.tsa.arima.model import ARIMA
# Linear Regression Model
from sklearn.linear_model import LinearRegression 
# to save model
import joblib
# use for standardization 
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

In [134]:
train = pd.read_csv('./train.csv')
train.drop(['종목명'],axis=1,inplace=True)
train['일자'] = pd.to_datetime(train['일자'], format='%Y%m%d')
#train = train[train['일자'].dt.year == 2023]
train = train[(train['일자'].dt.year == 2023) | (train['일자'].dt.year == 2022)]
train = train.reset_index(drop=True)
train = train[train['종목코드'] == "A050120"]
train

,일자,종목코드,거래량,시가,고가,저가,종가
47,2022-01-03,A050120,31183,4920,4960,4870,4890
2047,2022-01-04,A050120,15433,4925,4950,4850,4905
4047,2022-01-05,A050120,13292,4905,4910,4830,4875
6047,2022-01-06,A050120,17778,4825,4845,4750,4815
8047,2022-01-07,A050120,13304,4790,4915,4790,4840
...,...,...,...,...,...,...,...
684047,2023-05-23,A050120,59606,4795,4830,4670,4670
686047,2023-05-24,A050120,54899,4670,4750,4550,4645
688047,2023-05-25,A050120,70296,4620,4695,4460,4485
690047,2023-05-26,A050120,79944,4505,4620,4430,4565


In [135]:
out = pd.read_parquet('train_finance.parquet')
out = out[(out['date'].dt.year == 2023) | (out['date'].dt.year == 2022)]
out['ticker'] = 'A' + out['ticker']
out = out[out['ticker'] == "A050120"] 
train = train.reset_index(drop = True)
out = out.reset_index(drop = True)
z = pd.concat([train,out],axis=1)
z.drop(['date','ticker','name','volume','open','high','low','close'],axis=1,inplace=True)
train = z.copy()
train

,일자,종목코드,거래량,시가,고가,저가,종가,BPS,PER,PBR,...,total,market_cap,trading_volume,trading_value,shares_outstanding,shares_outstanding.1,holding_volume,exhaustion_rate,limit_exhaustion_volume,limit_exhaustion_rate
0,2022-01-03,A050120,31183,4920,4960,4870,4890,5120.0,0.00,0.96,...,0.0,6.632838e+10,31183.0,152706280.0,13564086.0,13564086.0,367243.0,2.71,13564086.0,2.71
1,2022-01-04,A050120,15433,4925,4950,4850,4905,5120.0,0.00,0.96,...,0.0,6.653184e+10,15433.0,75550160.0,13564086.0,13564086.0,369036.0,2.72,13564086.0,2.72
2,2022-01-05,A050120,13292,4905,4910,4830,4875,5120.0,0.00,0.95,...,0.0,6.612492e+10,13292.0,64663005.0,13564086.0,13564086.0,370600.0,2.73,13564086.0,2.73
3,2022-01-06,A050120,17778,4825,4845,4750,4815,5120.0,0.00,0.94,...,0.0,6.531107e+10,17778.0,85085660.0,13564086.0,13564086.0,370131.0,2.73,13564086.0,2.73
4,2022-01-07,A050120,13304,4790,4915,4790,4840,5120.0,0.00,0.95,...,0.0,6.565018e+10,13304.0,64623560.0,13564086.0,13564086.0,369364.0,2.72,13564086.0,2.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,2023-05-23,A050120,59606,4795,4830,4670,4670,8242.0,2.43,0.57,...,0.0,6.334428e+10,59606.0,283473805.0,13564086.0,13564086.0,462922.0,3.41,13564086.0,3.41
343,2023-05-24,A050120,54899,4670,4750,4550,4645,8242.0,2.42,0.56,...,0.0,6.300518e+10,54899.0,253635855.0,13564086.0,13564086.0,458732.0,3.38,13564086.0,3.38
344,2023-05-25,A050120,70296,4620,4695,4460,4485,8242.0,2.34,0.54,...,0.0,6.083493e+10,70296.0,319058410.0,13564086.0,13564086.0,475456.0,3.51,13564086.0,3.51
345,2023-05-26,A050120,79944,4505,4620,4430,4565,8242.0,2.38,0.55,...,0.0,6.192005e+10,79944.0,360554970.0,13564086.0,13564086.0,484289.0,3.57,13564086.0,3.57


In [136]:

# 각 주식 종목에 대해 15일간의 종가를 구하기 위해 더 복잡한 접근법
for i in range(1, 16):
    train[f'종가_lag_{i}'] = train.groupby('종목코드')['종가'].shift(-i)

In [137]:
dropped_samples = pd.DataFrame()

print(train['종목코드'].nunique())

stocks_to_drop = train[train['시가'] == 0]['종목코드'].unique()
dropped_samples = pd.concat([dropped_samples, train[train['종목코드'].isin(stocks_to_drop)]])

train = train[~train['종목코드'].isin(stocks_to_drop)]

print(train['종목코드'].nunique())

code_counts = train['종목코드'].value_counts()

max_count = code_counts.max()

stocks_to_drop = code_counts[code_counts < max_count].index
dropped_samples = pd.concat([dropped_samples, train[train['종목코드'].isin(stocks_to_drop)]])

train = train[~train['종목코드'].isin(stocks_to_drop)]

print(train['종목코드'].nunique())

train['종가변동률'] = train.groupby('종목코드')['종가'].pct_change()

stocks_to_drop = train[train['종가변동률'] <= -0.5]['종목코드'].unique()
dropped_samples = pd.concat([dropped_samples, train[train['종목코드'].isin(stocks_to_drop)]])

train = train[~train['종목코드'].isin(stocks_to_drop)]
train.drop(['종가변동률'],axis=1,inplace=True)

print(train['종목코드'].nunique())
train['일자'] = pd.to_datetime(train['일자'])  # 날짜 형식이 맞는지 확인
train = train.sort_values(by=['종목코드', '일자'])  # 주식 종목과 날짜 순으로 정렬


# shift를 이용해 n일 전의 거래량을 가져옵니다.
for i in range(1, 16):
    train[f'거래량_lag_{i}'] = train.groupby('종목코드')['거래량'].transform(lambda x: x.shift(i)).fillna(0)

train['3일_거래량_증가_여부'] = train.apply(lambda row: 1 if all(row[f'거래량_lag_{i}'] < row[f'거래량_lag_{i+1}'] for i in range(1, 3)) else 0, axis=1)


train['7일_거래량_증가_여부'] = train.apply(lambda row: 1 if all(row[f'거래량_lag_{i}'] < row[f'거래량_lag_{i+1}'] for i in range(1, 7)) else 0, axis=1)

# 15일전부터 거래량이 꾸준히 상승하는지 판단하는 피처 생성
train['15일_거래량_증가_여부'] = train.apply(lambda row: 1 if all(row[f'거래량_lag_{i}'] < row[f'거래량_lag_{i+1}'] for i in range(1, 15)) else 0, axis=1)



# 각 주식 종목에 대해 15일간의 종가를 구하기 위해 더 복잡한 접근법
for i in range(1, 16):
    train[f'종가_lag_{i}'] = train.groupby('종목코드')['종가'].shift(-i)

train['3일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=3).mean())
train['5일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=5).mean())
train['7일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=7).mean())
train['10일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=10).mean())
train['15일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=15).mean())
#train['30일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=30).mean())


train['종가_대비_3일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['3일_지수이동평균'] else 0, axis=1)
train['종가_대비_5일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['5일_지수이동평균'] else 0, axis=1)
train['종가_대비_7일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['7일_지수이동평균'] else 0, axis=1)
train['종가_대비_10일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['10일_지수이동평균'] else 0, axis=1)
train['종가_대비_15일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['15일_지수이동평균'] else 0, axis=1)
#train['종가_대비_30일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['30일_지수이동평균'] else 0, axis=1)

train['종가_대비_3일_지수이동평균'] = train['종가'] - train['3일_지수이동평균']
train['종가_대비_5일_지수이동평균'] = train['종가'] - train['5일_지수이동평균']
train['종가_대비_7일_지수이동평균'] = train['종가'] - train['7일_지수이동평균']
train['종가_대비_10일_지수이동평균'] = train['종가'] - train['10일_지수이동평균']
train['종가_대비_15일_지수이동평균'] = train['종가'] - train['15일_지수이동평균']
#train['종가_대비_30일_지수이동평균'] = train['종가'] - train['30일_지수이동평균']



train['거래량_변화율'] = train.groupby('종목코드')['거래량'].pct_change()

train['3일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=3, min_periods=1).std())
train['5일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=5, min_periods=1).std())
train['7일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=7, min_periods=1).std())
train['10일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=10, min_periods=1).std())
train['15일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=15, min_periods=1).std())
#train['30일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=30, min_periods=1).std())
from scipy.signal import convolve

def weighted_moving_average(series, period, weights=None):
    if weights is None:
        return series.rolling(period).mean()
    else:
        return series.rolling(period).apply(lambda x: np.sum(weights*x), raw=True)

def hull_moving_average(series, period):
    wma_half = weighted_moving_average(series, int(period/2))
    wma_full = weighted_moving_average(series, period)
    sqrt_period = int(np.sqrt(period))
    series = 2 * wma_half - wma_full
    return weighted_moving_average(series, sqrt_period, weights=np.arange(1, sqrt_period + 1))

# Now apply the hull moving average to the data
train['3일_HMA'] = train.groupby('종목코드')['종가'].transform(lambda x: hull_moving_average(x, 3))
train['5일_HMA'] = train.groupby('종목코드')['종가'].transform(lambda x: hull_moving_average(x, 5))
train['7일_HMA'] = train.groupby('종목코드')['종가'].transform(lambda x: hull_moving_average(x, 7))
train['10일_HMA'] = train.groupby('종목코드')['종가'].transform(lambda x: hull_moving_average(x, 10))
train['15일_HMA'] = train.groupby('종목코드')['종가'].transform(lambda x: hull_moving_average(x, 15))


train


1
1
1
1


,일자,종목코드,거래량,시가,고가,저가,종가,BPS,PER,PBR,...,3일_변동성,5일_변동성,7일_변동성,10일_변동성,15일_변동성,3일_HMA,5일_HMA,7일_HMA,10일_HMA,15일_HMA
0,2022-01-03,A050120,31183,4920,4960,4870,4890,5120.0,0.00,0.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-04,A050120,15433,4925,4950,4850,4905,5120.0,0.00,0.96,...,10.606602,10.606602,10.606602,10.606602,10.606602,NaN,NaN,NaN,NaN,NaN
2,2022-01-05,A050120,13292,4905,4910,4830,4875,5120.0,0.00,0.95,...,15.000000,15.000000,15.000000,15.000000,15.000000,4860.000000,NaN,NaN,NaN,NaN
3,2022-01-06,A050120,17778,4825,4845,4750,4815,5120.0,0.00,0.94,...,45.825757,39.449335,39.449335,39.449335,39.449335,4765.000000,NaN,NaN,NaN,NaN
4,2022-01-07,A050120,13304,4790,4915,4790,4840,5120.0,0.00,0.95,...,30.138569,36.912058,36.912058,36.912058,36.912058,4836.666667,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,2023-05-23,A050120,59606,4795,4830,4670,4670,8242.0,2.43,0.57,...,80.052066,173.075128,257.029737,238.460922,210.632268,4588.333333,14437.0,14897.380952,29096.0,28195.047619
343,2023-05-24,A050120,54899,4670,4750,4550,4645,8242.0,2.42,0.56,...,100.374299,124.348301,209.438024,227.378783,205.621590,4575.000000,13851.0,14416.904762,29580.0,28609.476190
344,2023-05-25,A050120,70296,4620,4695,4460,4485,8242.0,2.34,0.54,...,100.374299,129.836436,171.207198,225.468032,203.394995,4370.000000,13451.0,13785.000000,29179.0,28947.904762
345,2023-05-26,A050120,79944,4505,4620,4430,4565,8242.0,2.38,0.55,...,80.000000,129.102672,158.534959,214.512367,202.976658,4565.000000,13275.0,13370.714286,28547.0,29152.666667


In [138]:
def split_data(df, val_days=0, test_days=30):
    """
    데이터프레임을 주어진 일수에 따라 훈련, 검증, 테스트 세트로 분할합니다.

    Parameters:
    df (pandas.DataFrame): 분할할 데이터프레임
    val_days (int): 검증 세트로 사용할 일수
    test_days (int): 테스트 세트로 사용할 일수

    Returns:
    train (pandas.DataFrame): 훈련 세트
    val (pandas.DataFrame): 검증 세트
    test (pandas.DataFrame): 테스트 세트
    """
    df = df.sort_values(by=['종목코드', '일자'])

    df['rank'] = df.groupby('종목코드')['일자'].rank(method='first', ascending=False)

    train = df[df['rank'] > (val_days + test_days)]
    val = df[(df['rank'] > test_days) & (df['rank'] <= (val_days + test_days))]
    test = df[df['rank'] <= test_days]

    return train, val, test
train, _, test = split_data(train, val_days=0, test_days=30)

#train = train.dropna()
#test = test.dropna()

train = train.dropna()
train_x = train.drop(train.filter(regex='lag').columns, axis=1)
train_x.drop(['rank'],axis=1,inplace=True)
test_x = test.drop(test.filter(regex='lag').columns, axis=1)
test_x.drop(['rank'],axis=1,inplace=True)


train_y = train[train.filter(regex='lag').columns]
train_y = pd.concat([train['종목코드'], train_y],axis=1)
test_y = test['종목코드']
train_y

,종목코드,종가_lag_1,종가_lag_2,종가_lag_3,종가_lag_4,종가_lag_5,종가_lag_6,종가_lag_7,종가_lag_8,종가_lag_9,...,거래량_lag_6,거래량_lag_7,거래량_lag_8,거래량_lag_9,거래량_lag_10,거래량_lag_11,거래량_lag_12,거래량_lag_13,거래량_lag_14,거래량_lag_15
16,A050120,4965.0,4845.0,5200.0,5750.0,5820.0,5760.0,5800.0,5750.0,5690.0,...,27192.0,20780.0,11297.0,24148.0,52188.0,15132.0,13304.0,17778.0,13292.0,15433.0
17,A050120,4845.0,5200.0,5750.0,5820.0,5760.0,5800.0,5750.0,5690.0,5790.0,...,614610.0,27192.0,20780.0,11297.0,24148.0,52188.0,15132.0,13304.0,17778.0,13292.0
18,A050120,5200.0,5750.0,5820.0,5760.0,5800.0,5750.0,5690.0,5790.0,5690.0,...,142194.0,614610.0,27192.0,20780.0,11297.0,24148.0,52188.0,15132.0,13304.0,17778.0
19,A050120,5750.0,5820.0,5760.0,5800.0,5750.0,5690.0,5790.0,5690.0,5390.0,...,254553.0,142194.0,614610.0,27192.0,20780.0,11297.0,24148.0,52188.0,15132.0,13304.0
20,A050120,5820.0,5760.0,5800.0,5750.0,5690.0,5790.0,5690.0,5390.0,5170.0,...,165376.0,254553.0,142194.0,614610.0,27192.0,20780.0,11297.0,24148.0,52188.0,15132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,A050120,4375.0,4305.0,4390.0,4460.0,4600.0,4470.0,4465.0,4520.0,4450.0,...,10860.0,21954.0,11732.0,16019.0,50015.0,44783.0,17836.0,18902.0,9634.0,20648.0
313,A050120,4305.0,4390.0,4460.0,4600.0,4470.0,4465.0,4520.0,4450.0,4325.0,...,10963.0,10860.0,21954.0,11732.0,16019.0,50015.0,44783.0,17836.0,18902.0,9634.0
314,A050120,4390.0,4460.0,4600.0,4470.0,4465.0,4520.0,4450.0,4325.0,4250.0,...,122232.0,10963.0,10860.0,21954.0,11732.0,16019.0,50015.0,44783.0,17836.0,18902.0
315,A050120,4460.0,4600.0,4470.0,4465.0,4520.0,4450.0,4325.0,4250.0,4135.0,...,23285.0,122232.0,10963.0,10860.0,21954.0,11732.0,16019.0,50015.0,44783.0,17836.0


In [139]:
pred_df = test_x[['종목코드']]
pred_df

,종목코드
317,A050120
318,A050120
319,A050120
320,A050120
321,A050120
322,A050120
323,A050120
324,A050120
325,A050120
326,A050120


In [140]:
# '일자' 컬럼을 datetime 형식으로 변환
test_x['일자'] = pd.to_datetime(test_x['일자'])

# '2023-05-09' 이후의 데이터만 필터링하여 새로운 데이터프레임에 복사
new_df = test_x[test_x['일자'] >= '2023-05-09'].copy()
new_df

,일자,종목코드,거래량,시가,고가,저가,종가,BPS,PER,PBR,...,3일_변동성,5일_변동성,7일_변동성,10일_변동성,15일_변동성,3일_HMA,5일_HMA,7일_HMA,10일_HMA,15일_HMA
332,2023-05-09,A050120,13729,4355,4400,4355,4355,8242.0,2.27,0.53,...,52.519838,67.175144,124.187838,121.450722,131.432583,4303.333333,13264.0,13527.857143,26623.5,25568.476190
333,2023-05-10,A050120,11094,4320,4365,4300,4330,8242.0,2.26,0.53,...,38.188131,50.124844,71.672204,121.633557,126.232024,4296.666667,12970.0,13233.809524,26860.0,26004.238095
334,2023-05-11,A050120,67627,4335,4545,4330,4450,8242.0,2.32,0.54,...,63.311400,57.118298,65.054922,117.312498,125.126603,4521.666667,13055.0,13118.571429,26884.5,26386.523810
335,2023-05-12,A050120,14847,4450,4455,4345,4380,8242.0,2.28,0.53,...,60.277138,46.287147,47.245559,109.919162,114.498908,4373.333333,13272.0,13126.190476,26723.5,26688.714286
336,2023-05-15,A050120,23926,4345,4380,4270,4275,8242.0,2.23,0.52,...,88.081402,64.478679,65.918420,72.012730,108.865360,4181.666667,13040.0,13090.952381,26400.5,26747.666667
337,2023-05-16,A050120,40750,4240,4405,4230,4325,8242.0,2.25,0.52,...,52.519838,66.200453,57.590508,64.584140,108.865360,4323.333333,12793.0,12944.047619,26154.0,26621.761905
338,2023-05-17,A050120,56400,4405,4500,4325,4490,8242.0,2.34,0.54,...,112.509259,87.991477,74.992063,67.585337,117.304041,4616.666667,13110.0,13002.380952,26175.5,26545.428571
339,2023-05-18,A050120,531508,4500,4995,4500,4950,8242.0,2.58,0.60,...,323.895559,272.474769,229.779915,190.805311,190.344300,5311.666667,14343.0,13793.571429,26678.0,26870.428571
340,2023-05-19,A050120,113180,4970,4975,4645,4755,8242.0,2.48,0.58,...,230.885975,287.867157,246.151328,215.149173,206.782034,4778.333333,15248.0,14610.476190,27372.5,27363.333333
341,2023-05-22,A050120,50180,4805,4895,4715,4830,8242.0,2.52,0.59,...,98.361578,256.246951,269.488051,240.979483,212.248170,4815.000000,14976.0,15181.190476,28297.0,27825.571429


In [141]:
# Linear Regression Model
from sklearn.linear_model import LinearRegression 
# to save model
import joblib
# use for standardization 
from scipy import stats

In [142]:
test_x = test_x[test_x['일자'] == '2023-05-08']
test_x

,일자,종목코드,거래량,시가,고가,저가,종가,BPS,PER,PBR,...,3일_변동성,5일_변동성,7일_변동성,10일_변동성,15일_변동성,3일_HMA,5일_HMA,7일_HMA,10일_HMA,15일_HMA
331,2023-05-08,A050120,20058,4500,4530,4390,4405,8242.0,2.3,0.53,...,33.291641,86.631403,130.844328,119.848284,147.928554,4388.333333,13570.0,13587.380952,26147.5,25169.666667


In [143]:
pred_df = test_x[['종목코드']]
pred_df

,종목코드
331,A050120


In [144]:
train_x.drop(['일자','종목코드'],axis=1,inplace=True)
test_x.drop(['일자','종목코드'],axis=1,inplace=True)
train_x.drop(['시가','고가', '저가'],axis=1,inplace=True)
test_x.drop(['시가','고가', '저가'],axis=1,inplace=True)
train_x = train_x.reset_index(drop = True)
train_y = train_y.reset_index(drop = True)
test_x = test_x.reset_index(drop = True)
test_x

,거래량,종가,BPS,PER,PBR,EPS,DIV,DPS,financial_investment,insurance,...,3일_변동성,5일_변동성,7일_변동성,10일_변동성,15일_변동성,3일_HMA,5일_HMA,7일_HMA,10일_HMA,15일_HMA
0,20058,4405,8242.0,2.3,0.53,1918.0,0.0,0.0,0.0,0.0,...,33.291641,86.631403,130.844328,119.848284,147.928554,4388.333333,13570.0,13587.380952,26147.5,25169.666667


In [145]:
train_y.drop(['종목코드'],axis=1,inplace=True)
train_y

,종가_lag_1,종가_lag_2,종가_lag_3,종가_lag_4,종가_lag_5,종가_lag_6,종가_lag_7,종가_lag_8,종가_lag_9,종가_lag_10,...,거래량_lag_6,거래량_lag_7,거래량_lag_8,거래량_lag_9,거래량_lag_10,거래량_lag_11,거래량_lag_12,거래량_lag_13,거래량_lag_14,거래량_lag_15
0,4965.0,4845.0,5200.0,5750.0,5820.0,5760.0,5800.0,5750.0,5690.0,5790.0,...,27192.0,20780.0,11297.0,24148.0,52188.0,15132.0,13304.0,17778.0,13292.0,15433.0
1,4845.0,5200.0,5750.0,5820.0,5760.0,5800.0,5750.0,5690.0,5790.0,5690.0,...,614610.0,27192.0,20780.0,11297.0,24148.0,52188.0,15132.0,13304.0,17778.0,13292.0
2,5200.0,5750.0,5820.0,5760.0,5800.0,5750.0,5690.0,5790.0,5690.0,5390.0,...,142194.0,614610.0,27192.0,20780.0,11297.0,24148.0,52188.0,15132.0,13304.0,17778.0
3,5750.0,5820.0,5760.0,5800.0,5750.0,5690.0,5790.0,5690.0,5390.0,5170.0,...,254553.0,142194.0,614610.0,27192.0,20780.0,11297.0,24148.0,52188.0,15132.0,13304.0
4,5820.0,5760.0,5800.0,5750.0,5690.0,5790.0,5690.0,5390.0,5170.0,5390.0,...,165376.0,254553.0,142194.0,614610.0,27192.0,20780.0,11297.0,24148.0,52188.0,15132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,4375.0,4305.0,4390.0,4460.0,4600.0,4470.0,4465.0,4520.0,4450.0,4325.0,...,10860.0,21954.0,11732.0,16019.0,50015.0,44783.0,17836.0,18902.0,9634.0,20648.0
297,4305.0,4390.0,4460.0,4600.0,4470.0,4465.0,4520.0,4450.0,4325.0,4250.0,...,10963.0,10860.0,21954.0,11732.0,16019.0,50015.0,44783.0,17836.0,18902.0,9634.0
298,4390.0,4460.0,4600.0,4470.0,4465.0,4520.0,4450.0,4325.0,4250.0,4135.0,...,122232.0,10963.0,10860.0,21954.0,11732.0,16019.0,50015.0,44783.0,17836.0,18902.0
299,4460.0,4600.0,4470.0,4465.0,4520.0,4450.0,4325.0,4250.0,4135.0,4200.0,...,23285.0,122232.0,10963.0,10860.0,21954.0,11732.0,16019.0,50015.0,44783.0,17836.0


In [65]:
import itertools
lock = ['거래량', '종가']

additional_data = train_x.columns  # 실제 추가할 데이터의 이름을 넣어주세요
#additional_data = ['PI23102', 'TI23129', 'dff_1', 'FIC23111','TI23502']
# 실제 추가할 데이터의 이름을 넣어주세요

additional_data.drop('거래량')
additional_data.drop('종가')

for i in range(7, 9):  # 최대 5개까지의 추가 데이터
    for combination in itertools.combinations(additional_data, i):
        new_input = lock + list(combination)
        print(new_input)
        train_x1 = train_x[new_input].copy()
        test_x1 = test_x[new_input].copy()

        model = LinearRegression()
        model.fit(train_x1, train_y['종가_lag_1'])

        pred_1 = model.predict(test_x1)

        model = LinearRegression()
        model.fit(train_x1, train_y['종가_lag_15'])

        pred_15 = model.predict(test_x1)
        print('Combination:', combination)
        print('Target_1:', pred_1, 'Target_15:', pred_15)


['거래량', '종가', '거래량', '종가', '3일_거래량_증가_여부', '7일_거래량_증가_여부', '15일_거래량_증가_여부', '3일_지수이동평균', '5일_지수이동평균']
Combination: ('거래량', '종가', '3일_거래량_증가_여부', '7일_거래량_증가_여부', '15일_거래량_증가_여부', '3일_지수이동평균', '5일_지수이동평균')
Target_1: [81662.71651802] Target_15: [81900.62894422]
['거래량', '종가', '거래량', '종가', '3일_거래량_증가_여부', '7일_거래량_증가_여부', '15일_거래량_증가_여부', '3일_지수이동평균', '7일_지수이동평균']
Combination: ('거래량', '종가', '3일_거래량_증가_여부', '7일_거래량_증가_여부', '15일_거래량_증가_여부', '3일_지수이동평균', '7일_지수이동평균')
Target_1: [81663.39185821] Target_15: [82170.03336858]
['거래량', '종가', '거래량', '종가', '3일_거래량_증가_여부', '7일_거래량_증가_여부', '15일_거래량_증가_여부', '3일_지수이동평균', '10일_지수이동평균']
Combination: ('거래량', '종가', '3일_거래량_증가_여부', '7일_거래량_증가_여부', '15일_거래량_증가_여부', '3일_지수이동평균', '10일_지수이동평균')
Target_1: [81652.44772486] Target_15: [82365.89854808]
['거래량', '종가', '거래량', '종가', '3일_거래량_증가_여부', '7일_거래량_증가_여부', '15일_거래량_증가_여부', '3일_지수이동평균', '15일_지수이동평균']
Combination: ('거래량', '종가', '3일_거래량_증가_여부', '7일_거래량_증가_여부', '15일_거래량_증가_여부', '3일_지수이동평균', '15일_지수이동평균')
Target_1: [8164

KeyboardInterrupt: 

In [146]:

models = []
for i in range(1, 16):
    model = LinearRegression()
    model.fit(train_x, train_y[f'종가_lag_{i}'])
    models.append(model)


predictions = pd.DataFrame()
for i, model in enumerate(models, start=1):
    predictions[f'target_{i}'] = model.predict(test_x)
predictions

,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15
0,3476.11511,29.994142,-2251.302287,-3358.89926,-3326.514128,-2972.675617,-2748.528534,-2140.296508,-2642.411134,606.998607,1859.069563,2592.977082,980.915229,-1023.555589,-2094.255493


In [147]:
new_df['종가']

332    4355
333    4330
334    4450
335    4380
336    4275
337    4325
338    4490
339    4950
340    4755
341    4830
342    4670
343    4645
344    4485
345    4565
346    4595
Name: 종가, dtype: int64

In [103]:
pred_df['trend'] = 0
pred_df.loc[predictions['target_15'].values > predictions['target_1'].values, 'trend'] = 1
pred_df = pred_df.reset_index(drop = True)
pred_df['등락률'] = (((predictions['target_15'] - predictions['target_1']) / predictions['target_1']) * 100)
pred_df

,종목코드,trend,등락률
0,A037370,1,42.108537


# XGB

In [148]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
models = []
for i in range(1, 16):
    model = XGBRegressor(tree_method='gpu_hist', gpu_id=0, seed = 222)
    model.fit(train_x, train_y[f'종가_lag_{i}'])
    models.append(model)


predictions = pd.DataFrame()
for i, model in enumerate(models, start=1):
    predictions[f'target_{i}'] = model.predict(test_x)
predictions

,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15
0,4422.73291,4361.033203,4356.126465,4025.763428,4345.368164,4473.652832,4270.662598,4521.770996,4480.493652,4217.960449,4339.363281,4107.351562,4219.40918,4115.714355,4274.347656


In [149]:
new_df['종가']

332    4355
333    4330
334    4450
335    4380
336    4275
337    4325
338    4490
339    4950
340    4755
341    4830
342    4670
343    4645
344    4485
345    4565
346    4595
Name: 종가, dtype: int64

# Gluon

In [150]:

import os
import torch
import random
import numpy as np
import pandas as pd

from autogluon.tabular import TabularDataset, TabularPredictor

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate, train_test_split

from sklearn.model_selection import train_test_split

#train_data = TabularDataset(train)
test_data = TabularDataset(test_x)

models = []
for i in range(1, 16):
    train = pd.concat([train_x,train_y[f'종가_lag_{i}']], axis = 1)
    train_data = TabularDataset(train)
    predictor = TabularPredictor(label=f'종가_lag_{i}',  eval_metric='root_mean_squared_error').fit(train_data, presets='best_quality',  ag_args_fit={'num_gpus': 0})
    models.append(predictor)


predictions = pd.DataFrame()
for i, model in enumerate(models, start=1):
    predictions[f'target_{i}'] = model.predict(test_data)
predictions

No path specified. Models will be saved in: "AutogluonModels\ag-20230713_064514\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=5, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230713_064514\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   87.39 GB / 511.09 GB (17.1%)
Train Data Rows:    301
Train Data Columns: 58
Label Column: 종가_lag_1
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (6280.0, 3505.0, 4540.39867, 655.25784)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regr

,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15
0,4426.0,4667.347168,4787.233398,4925.678711,5248.549805,5300.557617,5257.753906,4939.833984,4817.683594,4912.840332,5084.329102,4983.333496,5168.058594,5133.116699,4952.685547


In [151]:
new_df['종가']

332    4355
333    4330
334    4450
335    4380
336    4275
337    4325
338    4490
339    4950
340    4755
341    4830
342    4670
343    4645
344    4485
345    4565
346    4595
Name: 종가, dtype: int64

# LGBM

In [152]:
from lightgbm import LGBMRegressor
models = []
for i in range(1, 16):
    model = LGBMRegressor(device='gpu', random_state=222)
    model.fit(train_x, train_y[f'종가_lag_{i}'])
    models.append(model)

predictions = pd.DataFrame()
for i, model in enumerate(models, start=1):
    predictions[f'target_{i}'] = model.predict(test_x)
predictions

,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15
0,4198.315204,4221.247687,4217.6232,4214.670999,4089.891547,4410.990367,4274.103926,4546.91913,4559.936614,4433.678267,4341.673939,4334.45961,4271.046089,4160.581384,4215.611864


In [153]:
new_df['종가']

332    4355
333    4330
334    4450
335    4380
336    4275
337    4325
338    4490
339    4950
340    4755
341    4830
342    4670
343    4645
344    4485
345    4565
346    4595
Name: 종가, dtype: int64

# Cat

In [154]:
from catboost import CatBoostRegressor
models = []
for i in range(1, 16):
    model = CatBoostRegressor(task_type='GPU', random_seed=222, verbose=False)
    model.fit(train_x, train_y[f'종가_lag_{i}'])
    models.append(model)

predictions = pd.DataFrame()
for i, model in enumerate(models, start=1):
    predictions[f'target_{i}'] = model.predict(test_x)
predictions

,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15
0,4397.030162,4327.238066,4251.672908,4275.655746,4275.956098,4294.369496,4277.909144,4333.776543,4319.672725,4274.375111,4287.184148,4253.739252,4370.828973,4259.491129,4300.354488


In [155]:
new_df['종가']

332    4355
333    4330
334    4450
335    4380
336    4275
337    4325
338    4490
339    4950
340    4755
341    4830
342    4670
343    4645
344    4485
345    4565
346    4595
Name: 종가, dtype: int64

# Metric

In [15]:
answer_df = pd.read_csv('Answer_real.csv', encoding = "CP949")
answer_df = answer_df[['종목코드','등락률']]
answer_df = answer_df.sort_values('종목코드')
# '종목코드' 피처에서 맨 앞 글자 제거
#answer_df['종목코드'] = answer_df['종목코드'].str[1:]
answer_df
answer_df['trend'] = 0
answer_df['trend'][answer_df['등락률'] > 0] = 1
answer_df.drop(['등락률'],axis=1,inplace=True)
answer_df

,종목코드,trend
334,A000020,1
1861,A000040,0
1358,A000050,0
764,A000070,1
1256,A000080,0
...,...,...
1077,A375500,0
1355,A378850,0
1294,A383220,0
683,A383310,1


In [994]:
#prediction_df = pd.read_csv('submission.csv')
#prediction_df['등락률'] = np.where(prediction_df['순위'] <= 1000, 1, -1)
#prediction_df

In [16]:
prediction_df = pred_df.reset_index(drop = True)
prediction_df

,종목코드,trend,등락률
0,A000020,0,-0.225380
1,A000040,1,4.015491
2,A000050,0,-1.581672
3,A000070,0,-0.920294
4,A000080,0,-1.058963
...,...,...,...
1807,A369370,0,-2.332737
1808,A373200,1,4.395897
1809,A378850,1,1.039643
1810,A383310,1,0.369173


In [17]:
answer_df = answer_df[answer_df['종목코드'].isin(prediction_df['종목코드'])]
answer_df = answer_df.reset_index(drop = True)
answer_df

,종목코드,trend
0,A000020,1
1,A000040,0
2,A000050,0
3,A000070,1
4,A000080,0
...,...,...
1807,A369370,0
1808,A373200,1
1809,A378850,0
1810,A383310,1


In [19]:
from sklearn.metrics import accuracy_score, mean_absolute_error

total_accuracy = accuracy_score(answer_df['trend'], prediction_df['trend'])
total_accuracy

0.4828918322295806

# Sub

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission

In [ ]:
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission

# Feature Selection

In [1127]:
answer_df = pd.read_csv('Answer_real.csv', encoding = "CP949")
answer_df = answer_df[['종목코드','등락률']]
answer_df = answer_df.sort_values('종목코드')
# '종목코드' 피처에서 맨 앞 글자 제거
#answer_df['종목코드'] = answer_df['종목코드'].str[1:]
answer_df
answer_df['trend'] = 0
answer_df['trend'][answer_df['등락률'] > 0] = 1
answer_df.drop(['등락률'],axis=1,inplace=True)
answer_df

,종목코드,trend
334,A000020,1
1861,A000040,0
1358,A000050,0
764,A000070,1
1256,A000080,0
...,...,...
1077,A375500,0
1355,A378850,0
1294,A383220,0
683,A383310,1


In [1128]:
prediction_df = pred_df.reset_index(drop = True)
prediction_df

,종목코드,trend,등락률
0,A000020,1,0.034149
1,A000040,1,11.121553
2,A000050,0,-1.755347
3,A000070,0,-1.056301
4,A000080,0,-0.913914
...,...,...,...
1807,A369370,0,-0.952087
1808,A373200,1,4.250982
1809,A378850,1,1.379645
1810,A383310,0,-0.139479


In [1129]:
answer_df = answer_df[answer_df['종목코드'].isin(prediction_df['종목코드'])]
answer_df = answer_df.reset_index(drop = True)
answer_df

,종목코드,trend
0,A000020,1
1,A000040,0
2,A000050,0
3,A000070,1
4,A000080,0
...,...,...
1807,A369370,0
1808,A373200,1
1809,A378850,0
1810,A383310,1


In [1130]:
from sklearn.metrics import accuracy_score, mean_absolute_error

total_accuracy = accuracy_score(answer_df['trend'], prediction_df['trend'])
total_accuracy

0.48233995584988965

In [1131]:
import itertools
lock = ['거래량', '종가']

additional_data = train_x.columns  # 실제 추가할 데이터의 이름을 넣어주세요
#additional_data = ['PI23102', 'TI23129', 'dff_1', 'FIC23111','TI23502']
# 실제 추가할 데이터의 이름을 넣어주세요

additional_data.drop('거래량')
additional_data.drop('종가')

non_df = prediction_df.copy()
non_df.drop(['trend'],axis=1,inplace=True)

for i in range(1, 5):  # 최대 5개까지의 추가 데이터
    for combination in itertools.combinations(additional_data, i):
        new_input = lock + list(combination)
        train_x1 = train_x[new_input].copy()
        test_x1 = test_x[new_input].copy()

        model = LinearRegression()
        model.fit(train_x1, train_y['종가_lag_1'])

        pred_1 = model.predict(test_x1)

        model = LinearRegression()
        model.fit(train_x1, train_y['종가_lag_15'])

        pred_15 = model.predict(test_x1)

        non_df['trend'] = 0
        non_df.loc[pred_15 > pred_1, 'trend'] = 1
        total_accuracy = accuracy_score(answer_df['trend'], non_df['trend'])
        if(total_accuracy > 0.51):
            print('Combination:', combination)
            print('Acc:', total_accuracy)


Combination: ('거래량', '종가_대비_15일_지수이동평균1')
Acc: 0.5110375275938189
Combination: ('7일_거래량_증가_여부', '종가_대비_15일_지수이동평균1')
Acc: 0.510485651214128
Combination: ('7일_지수이동평균', '종가_대비_15일_지수이동평균1')
Acc: 0.510485651214128
Combination: ('종가_대비_15일_지수이동평균1', '종가_대비_7일_지수이동평균')
Acc: 0.510485651214128
Combination: ('종가_대비_15일_지수이동평균1', '거래량_변화율')
Acc: 0.510485651214128
Combination: ('종가_대비_15일_지수이동평균1', '3일_변동성')
Acc: 0.5110375275938189
Combination: ('거래량', '7일_거래량_증가_여부', '종가_대비_15일_지수이동평균1')
Acc: 0.5115894039735099
Combination: ('거래량', '15일_거래량_증가_여부', '종가_대비_15일_지수이동평균1')
Acc: 0.5110375275938189
Combination: ('거래량', '5일_지수이동평균', '종가_대비_15일_지수이동평균1')
Acc: 0.5110375275938189
Combination: ('거래량', '7일_지수이동평균', '종가_대비_15일_지수이동평균1')
Acc: 0.510485651214128
Combination: ('거래량', '종가_대비_15일_지수이동평균1', '종가_대비_7일_지수이동평균')
Acc: 0.5110375275938189
Combination: ('거래량', '종가_대비_15일_지수이동평균1', '3일_변동성')
Acc: 0.5121412803532008
Combination: ('거래량', '종가_대비_15일_지수이동평균1', '5일_변동성')
Acc: 0.5110375275938189
Combination: ('

: 

# Feature Selection2

In [1097]:
answer_df = pd.read_csv('Answer_real.csv', encoding = "CP949")
answer_df = answer_df[['종목코드','등락률']]
answer_df = answer_df.sort_values('종목코드')
# '종목코드' 피처에서 맨 앞 글자 제거
#answer_df['종목코드'] = answer_df['종목코드'].str[1:]
answer_df['trend'] = 0
answer_df['trend'][answer_df['등락률'] > 0] = 1
#answer_df.drop(['등락률'],axis=1,inplace=True)
answer_df

,종목코드,등락률,trend
334,A000020,8.04,1
1861,A000040,-12.17,0
1358,A000050,-3.76,0
764,A000070,0.97,1
1256,A000080,-2.89,0
...,...,...,...
1077,A375500,-1.24,0
1355,A378850,-3.74,0
1294,A383220,-3.20,0
683,A383310,1.77,1


In [1098]:
prediction_df = pd.read_csv('prepro.csv')
prediction_df['trend'] = np.where(prediction_df['순위'] <= 1000, 1, 0)
prediction_df

,종목코드,순위,trend
0,A000020,361.0,1
1,A000040,849.0,1
2,A000050,1000.0,1
3,A000070,81.0,1
4,A000080,642.0,1
...,...,...,...
1995,A375500,1754.0,0
1996,A378850,1765.0,0
1997,A383220,1755.0,0
1998,A383310,396.0,1


In [1099]:
prediction_df = pred_df.reset_index(drop = True)
prediction_df

,종목코드,trend,등락률
0,A000020,1,0.525206
1,A000040,1,25.049831
2,A000050,1,0.128752
3,A000070,0,-1.360945
4,A000080,0,-0.804399
...,...,...,...
1995,A375500,0,-1.102470
1996,A378850,1,2.709759
1997,A383220,0,-1.479907
1998,A383310,0,-1.330883


In [1100]:
answer_df = answer_df[answer_df['종목코드'].isin(prediction_df['종목코드'])]
answer_df = answer_df.reset_index(drop = True)
answer_df

,종목코드,등락률,trend
0,A000020,8.04,1
1,A000040,-12.17,0
2,A000050,-3.76,0
3,A000070,0.97,1
4,A000080,-2.89,0
...,...,...,...
1995,A375500,-1.24,0
1996,A378850,-3.74,0
1997,A383220,-3.20,0
1998,A383310,1.77,1


In [1101]:
answer_df = answer_df.sort_values('등락률')
top_200_ans = answer_df.head(200)
bottom_200_ans = answer_df.tail(200)
answer_df = pd.concat([top_200_ans, bottom_200_ans])

prediction_df = prediction_df.sort_values('등락률')
top_200_pred = prediction_df.head(200)
bottom_200_pred = prediction_df.tail(200)
prediction_df = pd.concat([top_200_pred, bottom_200_pred])

In [1102]:
top_acc = accuracy_score(top_200_pred['trend'], top_200_ans['trend'])
bot_acc = accuracy_score(bottom_200_pred['trend'], bottom_200_ans['trend'])
print(top_acc,bot_acc)

1.0 1.0


In [1103]:
from sklearn.metrics import accuracy_score, mean_absolute_error

total_accuracy = accuracy_score(answer_df['trend'], prediction_df['trend'])
total_accuracy

1.0

In [27]:
train

,일자,종목코드,거래량,시가,고가,저가,종가
44,2022-01-03,A017940,8021,47850,48150,47600,47650
2044,2022-01-04,A017940,7236,47650,48050,47600,47800
4044,2022-01-05,A017940,8176,47800,47850,47450,47750
6044,2022-01-06,A017940,8308,47450,48200,47400,47600
8044,2022-01-07,A017940,5917,47850,48050,47550,48000
...,...,...,...,...,...,...,...
684044,2023-05-23,A017940,16675,52500,53000,52100,52800
686044,2023-05-24,A017940,11486,52800,53000,52000,52600
688044,2023-05-25,A017940,7569,52800,52800,52100,52500
690044,2023-05-26,A017940,11113,52500,52700,51100,51200


In [34]:
z.columns

Index(['일자', '종목코드', '거래량', '시가', '고가', '저가', '종가', 'BPS', 'PER', 'PBR', 'EPS',
       'DIV', 'DPS', 'financial_investment', 'insurance', 'investment_trust',
       'private_equity', 'bank', 'other_finance', 'pension_fund',
       'other_corporations', 'individuals', 'foreigners', 'other_foreigners',
       'total', 'market_cap', 'trading_volume', 'trading_value',
       'shares_outstanding', 'shares_outstanding.1', 'holding_volume',
       'exhaustion_rate', 'limit_exhaustion_volume', 'limit_exhaustion_rate'],
      dtype='object')

In [33]:
out = pd.read_parquet('train_finance.parquet')
out = out[(out['date'].dt.year == 2023) | (out['date'].dt.year == 2022)]
out['ticker'] = 'A' + out['ticker']
out = out[out['ticker'] == "A017940"] 
train = train.reset_index(drop = True)
out = out.reset_index(drop = True)
z = pd.concat([train,out],axis=1)
z.drop(['date','ticker','name','volume','open','high','low','close'],axis=1,inplace=True)
z

,일자,종목코드,거래량,시가,고가,저가,종가,BPS,PER,PBR,...,total,market_cap,trading_volume,trading_value,shares_outstanding,shares_outstanding.1,holding_volume,exhaustion_rate,limit_exhaustion_volume,limit_exhaustion_rate
0,2022-01-03,A017940,8021,47850,48150,47600,47650,172701.0,4.57,0.28,...,0.0,3.268790e+11,8021.0,383119650.0,6860000.0,6860000.0,243277.0,3.55,6860000.0,3.55
1,2022-01-04,A017940,7236,47650,48050,47600,47800,172701.0,4.58,0.28,...,0.0,3.279080e+11,7236.0,346002650.0,6860000.0,6860000.0,243785.0,3.55,6860000.0,3.55
2,2022-01-05,A017940,8176,47800,47850,47450,47750,172701.0,4.58,0.28,...,0.0,3.275650e+11,8176.0,389040000.0,6860000.0,6860000.0,243714.0,3.55,6860000.0,3.55
3,2022-01-06,A017940,8308,47450,48200,47400,47600,172701.0,4.57,0.28,...,0.0,3.265360e+11,8308.0,398369450.0,6860000.0,6860000.0,242557.0,3.54,6860000.0,3.54
4,2022-01-07,A017940,5917,47850,48050,47550,48000,172701.0,4.60,0.28,...,0.0,3.292800e+11,5917.0,283579950.0,6860000.0,6860000.0,243000.0,3.54,6860000.0,3.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,2023-05-23,A017940,16675,52500,53000,52100,52800,249560.0,2.15,0.21,...,0.0,3.622080e+11,16675.0,877843100.0,6860000.0,6860000.0,299025.0,4.36,6860000.0,4.36
343,2023-05-24,A017940,11486,52800,53000,52000,52600,249560.0,2.14,0.21,...,0.0,3.608360e+11,11486.0,601324100.0,6860000.0,6860000.0,303612.0,4.43,6860000.0,4.43
344,2023-05-25,A017940,7569,52800,52800,52100,52500,249560.0,2.14,0.21,...,0.0,3.601500e+11,7569.0,396471700.0,6860000.0,6860000.0,304635.0,4.44,6860000.0,4.44
345,2023-05-26,A017940,11113,52500,52700,51100,51200,249560.0,2.09,0.21,...,0.0,3.512320e+11,11113.0,575014300.0,6860000.0,6860000.0,306424.0,4.47,6860000.0,4.47


In [9]:
train

,일자,종목코드,거래량,시가,고가,저가,종가
44,2022-01-03,A017940,8021,47850,48150,47600,47650
2044,2022-01-04,A017940,7236,47650,48050,47600,47800
4044,2022-01-05,A017940,8176,47800,47850,47450,47750
6044,2022-01-06,A017940,8308,47450,48200,47400,47600
8044,2022-01-07,A017940,5917,47850,48050,47550,48000
...,...,...,...,...,...,...,...
684044,2023-05-23,A017940,16675,52500,53000,52100,52800
686044,2023-05-24,A017940,11486,52800,53000,52000,52600
688044,2023-05-25,A017940,7569,52800,52800,52100,52500
690044,2023-05-26,A017940,11113,52500,52700,51100,51200


In [14]:
out = out[(out['date'].dt.year == 2023) | (out['date'].dt.year == 2022)]
out['ticker'] = 'A' + out['ticker']
z = pd.merge(out, train, left_on = 'ticker')
z

,date,ticker,name,volume,open,high,low,close,BPS,PER,...,total,market_cap,trading_volume,trading_value,shares_outstanding,shares_outstanding.1,holding_volume,exhaustion_rate,limit_exhaustion_volume,limit_exhaustion_rate
147,2022-01-03,A000020,동화약품,255857,14800,15000,14600,14950,11860.0,14.47,...,0.0,4.175755e+11,255857.0,3.780367e+09,27931470.0,27931470.0,661962.0,2.37,27931470.0,2.37
148,2022-01-04,A000020,동화약품,235805,15000,15000,14750,14900,11860.0,14.42,...,0.0,4.161789e+11,235805.0,3.507059e+09,27931470.0,27931470.0,692672.0,2.48,27931470.0,2.48
149,2022-01-05,A000020,동화약품,308187,14900,14900,14500,14550,11860.0,14.09,...,0.0,4.064029e+11,308187.0,4.502919e+09,27931470.0,27931470.0,691694.0,2.48,27931470.0,2.48
150,2022-01-06,A000020,동화약품,303389,14400,14600,14200,14250,11860.0,13.79,...,0.0,3.980234e+11,303389.0,4.353750e+09,27931470.0,27931470.0,666224.0,2.39,27931470.0,2.39
151,2022-01-07,A000020,동화약품,158132,14200,14650,14200,14600,11860.0,14.13,...,0.0,4.077995e+11,158132.0,2.288628e+09,27931470.0,27931470.0,661170.0,2.37,27931470.0,2.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,A383800,LX홀딩스,150364,8390,8390,8310,8330,21133.0,3.80,...,0.0,6.354181e+11,150364.0,1.252835e+09,76280690.0,76280690.0,7756282.0,10.17,76280690.0,10.17
987996,2023-05-24,A383800,LX홀딩스,122457,8310,8340,8280,8300,21133.0,3.79,...,0.0,6.331297e+11,122457.0,1.016281e+09,76280690.0,76280690.0,7754179.0,10.17,76280690.0,10.17
987997,2023-05-25,A383800,LX홀딩스,84241,8300,8310,8270,8310,21133.0,3.79,...,0.0,6.338925e+11,84241.0,6.986522e+08,76280690.0,76280690.0,7749913.0,10.16,76280690.0,10.16
987998,2023-05-26,A383800,LX홀딩스,126681,8300,8310,8270,8280,21133.0,3.78,...,0.0,6.316041e+11,126681.0,1.049454e+09,76280690.0,76280690.0,7737809.0,10.14,76280690.0,10.14
